In [ ]:
import sys

!sudo add-apt-repository ppa:saiarcot895/chromium-beta
!sudo apt remove chromium-browser
!sudo snap remove chromium
!sudo apt install chromium-browser

!pip3 install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

sys.path.insert(0, '/usr/lib/chromium-browser/chromedriver')

In [2]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service

options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

webdriver_service = Service('/usr/bin/chromedriver')
driver = webdriver.Chrome(service = webdriver_service, options = options)

In [3]:
from selenium.webdriver.common.keys import Keys

url = "https://ncov.kdca.go.kr/"
driver.get(url)
time.sleep(3) # 질병관리청 페이지 오픈

# driver.get(url)
# driver.page_source
# driver.find_element -> text, click()
"""
title1 = driver.find_element(By.XPATH, "/html/body/div/div[5]/div/div/div/div[4]/div/div[1]/div[1]/div[1]/table/tbody/tr[1]/td[4]/span").text
print(title1)

driver.find_element(By.XPATH, "/html/body/div/div[5]/div/div/div/div[1]/div[2]/div/ul/li[1]/a").click()
time.sleep(3) #새페이지 오픈

content = driver.find_element(By.XPATH, "/html/body/div/form/div/div/div/div/div[2]/div[2]/div/p[9]/strong").text
print(content)
"""

driver.find_element(By.XPATH, "/html/body/div/div[3]/div/div[2]/a[1]").click()
time.sleep(1) # 돋보기오픈

driver.find_element(By.XPATH, "/html/body/div/div[3]/div/div[2]/div[4]/form/div/input[1]").click()
time.sleep(1) # 검색input
time.sleep(2)

driver.find_element(By.XPATH, "/html/body/div/div[3]/div/div[2]/div[4]/form/div/input[1]").send_keys("단계")
driver.find_element(By.XPATH, "/html/body/div/div[3]/div/div[2]/div[4]/form/div/input[1]").send_keys(Keys.ENTER)
time.sleep(4) # 검색후 새페이지 오픈

title = driver.find_element(By.XPATH, "/html/body/div[4]/main/div/section/div/section[2]/section[3]/article[1]/a/div/h3").text
print(title)

# change TEST

(경북권질병대응센터국립포항검역소)지역내축제기간국민참여형감염병예방홍보실시(5.26.금)


In [10]:
from selenium.webdriver.common.keys import Keys

url = "https://map.kakao.com/"
driver.get(url)
time.sleep(3)

# 0. 길찾기 버튼 클릭
driver.find_element(By.XPATH, "/html/body/div[5]/div[1]/div/div/ul/li[2]").click()
time.sleep(3)

aptNm = "서대문구 힐스테이트신촌아파트"
target = "어린이집"

start = "/html/body/div[5]/div[2]/div[2]/div[1]/div/div[1]/div[2]/form/input[1]"
arrive = "/html/body/div[5]/div[2]/div[2]/div[1]/div/div[2]/div[2]/form/input[1]"
walkButton = "/html/body/div[5]/div[2]/div[2]/div[2]/div/a[3]"
detailButton = "/html/body/div[5]/div[2]/div[2]/div[5]/div[3]/div[1]/ul/li[1]/div[1]/a";
keyWord = "횡단보도"
pumYn = 0

# 1-1. 검색창 클릭 - 입력
driver.find_element(By.XPATH, start).click()
driver.find_element(By.XPATH, start).send_keys(aptNm)

# 1-2. Enter눌러 검색
driver.find_element(By.XPATH, start).send_keys(Keys.ENTER)
time.sleep(3)

# 2-1. 검색창 클릭 - 서대문구 충정아파트 주변 어린이집 입력
driver.find_element(By.XPATH, arrive).click()
driver.find_element(By.XPATH, arrive).send_keys(aptNm + " 주변 " + target)

# 2-2. Enter눌러 서대문구 충정아파트 검색
driver.find_element(By.XPATH, arrive).send_keys(Keys.ENTER)
time.sleep(3)

# 3. 도보버튼 클릭
driver.find_element(By.XPATH, walkButton).click()
time.sleep(3) # 검색input

# 4. 세부경로 조회
driver.find_element(By.XPATH, detailButton).click()
time.sleep(1)

timeText = driver.find_element(By.XPATH, "/html/body/div[5]/div[2]/div[2]/div[5]/div[3]/div[1]/ul/li[1]/div[1]/div/p/span[2]").text
routeText = driver.find_element(By.XPATH, "/html/body/div[5]/div[2]/div[2]/div[5]/div[3]/div[1]/ul/li[1]/div[2]/ul").text

if keyWord not in routeText:
  pumYn = 1

# 소요시간 출력
print(timeText)
# 모든 세부경로 출력
# print(routeText)
# 얼품아여부 출력
print("얼품아여부 : " + str(pumYn))


1.5km
얼품아여부 : 0
